In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import scipy.io as sio

In [2]:
def deal_data(data, marker):
    # 舍弃99标签以前的数据(包括99标签) 、 91号以后的标签(包括91号)
    index_99 = np.max(np.where(marker == 99)[0])
    index_91 = np.min(np.where(marker == 91)[0])
    marker = marker[index_99 + 1: index_91]
    data = data[index_99 + 1: index_91]
    return data, marker


class Data:
    def __init__(self):
        self.simple_path = "data_f5/5F-SubjectB-160311-5St-SGLHand-HFREQ.mat"
        self.source_data = None  # [-1, 22]
        self.source_marker = None  # [-1, 1]
        self.mean = None
        self.std = None
        self.create_data()
        self.simple()

    def create_data(self):
        data, marker = self.load_mat_data()
        data, self.source_marker = deal_data(data, marker)  # marker: [-1, 1]
        # 归一化
        self.mean = np.mean(data, axis=0)
        self.std = np.std(data, axis=0)
        self.source_data = (data - self.mean) / self.std  # [-1, 22]

    def simple(self):
        """
        simple : [-1, 8, 128, 22]
        """
        markers = np.reshape(self.source_marker, [-1])
        start_index = 0
        current_marker = None
        simple = None
        label = []
        for index, marker in enumerate(markers):  # marker: 0 or 1 or 2 or 3 or 4 or 5
            if current_marker is None:
                current_marker = marker
                continue
            if current_marker == marker:
                continue
            data = self.source_data[start_index: index]  # [-1, 22]
            num = len(data) // 128
            if num == 0:
                continue
            data = data[: num * 128]  # [-1, 22]
            data = np.reshape(data, [-1, 128, 22])  # [-1, 128, 22]
            data = np.transpose(data, [0, 2, 1])  # [-1, 22, 128]
            if simple is None:
                simple = data
            else:
                simple = np.concatenate((simple, data), axis=0)
            for i in range(len(data)):
                label.append(current_marker)
            # 更新参数
            start_index = index
            current_marker = marker
        return simple, label

    def load_mat_data(self):
        """
        加载数据
        :return:
        data: [-1, 22]
        marker:
        0 : 无操作;
        1 - 5 ：对应手指操作;
        91 ：between;
        92 ：end;
        99 ：start
        """
        file = sio.loadmat(self.simple_path)
        data = file["o"]["data"][0][0]  # [-1, 22]
        data = np.array(data).astype(np.float)

        marker = file["o"]["marker"][0][0]  # [-1, 1]
        marker = np.array(marker).astype(np.int)
        return data, marker

    def to_value(self, data):
        if self.mean is None:
            self.get_param()
        return data * self.std + self.mean

    def get_param(self):
        data, source_marker = self.load_mat_data()
        # 归一化
        self.mean = np.mean(data, axis=0)
        self.std = np.std(data, axis=0)

In [4]:

DS = Data()
simple, marker = DS.simple()
marker = np.array(marker)
# 舍弃0标签的数据
index_0 = np.where(marker == 0)[0]
simple = np.delete(simple, index_0, axis=0)
marker = np.delete(marker, index_0, axis=0)
for i in range(1, 6):
    index_list = np.where(marker == i)[0]
    print("{i}: ".format(i=i), float(len(index_list))/len(marker))
marker = np.reshape(marker, [-1, 1])

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 22, 128)]         0         
_________________________________________________________________
batch_normalization (BatchNo (None, 22, 128)           512       
_________________________________________________________________
dense (Dense)                (None, 22, 32)            4128      
_________________________________________________________________
batch_normalization_1 (Batch (None, 22, 32)            128       
_________________________________________________________________
dense_1 (Dense)              (None, 22, 8)             264       
_________________________________________________________________
flatten (Flatten)            (None, 176)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 176)              

In [6]:
from keras import regularizers

Using TensorFlow backend.


In [44]:
class DNN:
    def __init__(self):     
        inputs = keras.Input([22, 128], dtype=tf.float32)  # [-1, 22, 128]
        d1 = keras.layers.Dense(32, activation=tf.nn.relu,
                                kernel_regularizer=keras.regularizers.L1(0.005))(inputs)  # [-1, 22, 32]

        d2 = keras.layers.Dense(2, activation=tf.nn.relu,
                                kernel_regularizer=keras.regularizers.L1(0.005))(d1)  # [-1, 22, 8]

        f = keras.layers.Flatten()(d2)  # [-1, 176]

        d3 = keras.layers.Dense(32, activation=tf.nn.relu,
                                kernel_regularizer=keras.regularizers.L1(0.005))(f)  # [-1, 32]

        outputs = keras.layers.Dense(6, activation=tf.nn.softmax,
                                     kernel_regularizer=keras.regularizers.L1(0.005))(d3)  # [-1, 6]

        self.model = keras.Model(inputs=inputs, outputs=outputs)
        self.model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),  # lr = 0.001
                           loss=tf.keras.losses.sparse_categorical_crossentropy,  # 交叉熵
                           metrics=['accuracy'])  # 准确度
        self.model.summary()

In [45]:
dnn = DNN()
history = dnn.model.fit(simple,
                        marker,
                        batch_size=200,
                        epochs=200,
                        validation_split=0.2)

Model: "functional_41"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        [(None, 22, 128)]         0         
_________________________________________________________________
dense_80 (Dense)             (None, 22, 32)            4128      
_________________________________________________________________
dense_81 (Dense)             (None, 22, 2)             66        
_________________________________________________________________
flatten_20 (Flatten)         (None, 44)                0         
_________________________________________________________________
dense_82 (Dense)             (None, 32)                1440      
_________________________________________________________________
dense_83 (Dense)             (None, 6)                 198       
Total params: 5,832
Trainable params: 5,832
Non-trainable params: 0
___________________________________________________

Epoch 51/200
13/13 [==============================] - 0s 5ms/step - loss: 1.7153 - accuracy: 0.2629 - val_loss: 1.7209 - val_accuracy: 0.2621
Epoch 52/200
13/13 [==============================] - 0s 5ms/step - loss: 1.7141 - accuracy: 0.2601 - val_loss: 1.7168 - val_accuracy: 0.2605
Epoch 53/200
13/13 [==============================] - 0s 5ms/step - loss: 1.7110 - accuracy: 0.2613 - val_loss: 1.7142 - val_accuracy: 0.2605
Epoch 54/200
13/13 [==============================] - 0s 6ms/step - loss: 1.7091 - accuracy: 0.2640 - val_loss: 1.7127 - val_accuracy: 0.2574
Epoch 55/200
13/13 [==============================] - 0s 5ms/step - loss: 1.7066 - accuracy: 0.2578 - val_loss: 1.7091 - val_accuracy: 0.2590
Epoch 56/200
13/13 [==============================] - 0s 5ms/step - loss: 1.7039 - accuracy: 0.2637 - val_loss: 1.7058 - val_accuracy: 0.2574
Epoch 57/200
13/13 [==============================] - 0s 4ms/step - loss: 1.7023 - accuracy: 0.2637 - val_loss: 1.7036 - val_accuracy: 0.2590
Epoch 

13/13 [==============================] - 0s 5ms/step - loss: 1.6590 - accuracy: 0.2625 - val_loss: 1.6592 - val_accuracy: 0.2746
Epoch 109/200
13/13 [==============================] - 0s 5ms/step - loss: 1.6582 - accuracy: 0.2621 - val_loss: 1.6582 - val_accuracy: 0.2730
Epoch 110/200
13/13 [==============================] - 0s 4ms/step - loss: 1.6588 - accuracy: 0.2660 - val_loss: 1.6593 - val_accuracy: 0.2668
Epoch 111/200
13/13 [==============================] - 0s 5ms/step - loss: 1.6591 - accuracy: 0.2637 - val_loss: 1.6566 - val_accuracy: 0.2746
Epoch 112/200
13/13 [==============================] - 0s 5ms/step - loss: 1.6589 - accuracy: 0.2660 - val_loss: 1.6584 - val_accuracy: 0.2730
Epoch 113/200
13/13 [==============================] - 0s 5ms/step - loss: 1.6577 - accuracy: 0.2668 - val_loss: 1.6574 - val_accuracy: 0.2621
Epoch 114/200
13/13 [==============================] - 0s 5ms/step - loss: 1.6575 - accuracy: 0.2621 - val_loss: 1.6561 - val_accuracy: 0.2668
Epoch 115/200

13/13 [==============================] - 0s 5ms/step - loss: 1.6421 - accuracy: 0.2707 - val_loss: 1.6456 - val_accuracy: 0.2543
Epoch 166/200
13/13 [==============================] - 0s 5ms/step - loss: 1.6418 - accuracy: 0.2660 - val_loss: 1.6461 - val_accuracy: 0.2652
Epoch 167/200
13/13 [==============================] - 0s 4ms/step - loss: 1.6417 - accuracy: 0.2676 - val_loss: 1.6457 - val_accuracy: 0.2574
Epoch 168/200
13/13 [==============================] - 0s 5ms/step - loss: 1.6424 - accuracy: 0.2590 - val_loss: 1.6461 - val_accuracy: 0.2559
Epoch 169/200
13/13 [==============================] - 0s 5ms/step - loss: 1.6419 - accuracy: 0.2679 - val_loss: 1.6454 - val_accuracy: 0.2605
Epoch 170/200
13/13 [==============================] - 0s 5ms/step - loss: 1.6421 - accuracy: 0.2648 - val_loss: 1.6436 - val_accuracy: 0.2559
Epoch 171/200
13/13 [==============================] - 0s 5ms/step - loss: 1.6414 - accuracy: 0.2629 - val_loss: 1.6465 - val_accuracy: 0.2527
Epoch 172/200